# Model wrapping
![](wrapping.png)

In [ ]:
from matlapy import Matlapy
import matlapy.blocks
from pathlib import Path

In [ ]:
from importlib import reload
reload(matlapy.blocks)

In [ ]:
mat = Matlapy()

In [ ]:
mat.connect_first_matlab()

In [ ]:
mat.eng.cd(str(Path().absolute()))

In [ ]:
mdl = mat.Slx("example_model").new()

In [ ]:
refmdl = mat.Slx("mdl1")

In [ ]:
sub = mdl.subsystem("func")
sub.moveTo(100, 100)
model = sub.add_block(matlapy.blocks.Model).moveTo(100, 100)
model.set_param("ModelName", "mdl1")
for port_idx, port_name in enumerate(refmdl.inports):
    if port_name.endswith("_"):
        port_name = port_name[:-1]
    if port_name.startswith("$"):
        port_name = port_name[1:]
    if port_idx == 0:
        convert = sub.add_block(matlapy.blocks.Convert).alignTo(model, 0, 2, marginX=-10)
    else:
        convert = sub.add_block(matlapy.blocks.Convert).alignTo(convert, 6, 9, marginY=30)
    convert.connectTo(model, dstport=port_idx+1, srcport=1)
    ds = sub.add_block(matlapy.blocks.DataStoreRead, f"DSR_{port_name}").alignTo(convert, 3, 5, marginX=-10).connectTo(convert).set_param("DataStoreName", port_name)

for port_idx, port_name in enumerate(refmdl.outports):
    if port_name.endswith("_"):
        port_name = port_name[:-1]
    if port_name.startswith("$"):
        port_name = port_name[1:]
    if port_idx == 0:
        convert = sub.add_block(matlapy.blocks.Convert).alignTo(model, 2, 0, marginX=10)
    else:
        convert = sub.add_block(matlapy.blocks.Convert).alignTo(convert, 6, 9, marginY=30)
    convert.connectFrom(model, srcport=port_idx+1, dstport=1)
    ds = sub.add_block(matlapy.blocks.DataStoreWrite, f"DSW_{port_name}").alignTo(convert, 5, 3, marginX=10).connectFrom(convert).set_param("DataStoreName", port_name)
        